In [32]:
import duckdb
import leafmap
import pandas as pd
import geopandas as gpd

In [4]:
con = duckdb.connect()
con.install_extension("httpfs")
con.load_extension("httpfs")
con.install_extension("spatial")
con.load_extension("spatial")

In [5]:
con.sql("""
        CREATE TABLE IF NOT EXISTS cpad as
        SELECT * FROM ST_Read('C:/Users/vance/Downloads/cpadclean.csv')

""")
con.table("cpad")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌──────────┬──────────────────────┬──────────────────────┬───┬────────────┬────────────┬─────────────┬───────────────┐
│ OBJECTID │         NAME         │        AGENCY        │ … │ GAP4_acres │ GAP_tot_ac │ Shape__Area │ Shape__Length │
│ varchar  │       varchar        │       varchar        │   │  varchar   │  varchar   │   varchar   │    varchar    │
├──────────┼──────────────────────┼──────────────────────┼───┼────────────┼────────────┼─────────────┼───────────────┤
│ 1        │ El Corte de Madera…  │ Midpeninsula Regio…  │ … │ 0          │ 1          │ 11258.48438 │ 419.143937    │
│ 2        │ La Raggione Tot Lot  │ San Jose, City of    │ … │ 0.31       │ 0.31       │ 1974.441406 │ 184.4334246   │
│ 3        │ Shady Oaks Park      │ San Jose, City of    │ … │ 8          │ 8          │ 53018.16406 │ 998.8756168   │
│ 4        │ West Evergreen Park  │ San Jose, City of    │ … │ 1          │ 1          │ 6596.527344 │ 329.6446295   │
│ 5        │ Martin Griffin Pre…  │ Audubon Cany

In [59]:
con.sql("""
        SELECT NAME, AGENCY, ACCESS, COUNTY, ACRES FROM CPAD
        ORDER BY ACRES desc
        LIMIT 20
""")


┌──────────────────────┬───────────────────────────────────────────┬───────────────────┬─────────────────┬─────────────┐
│         NAME         │                  AGENCY                   │      ACCESS       │     COUNTY      │    ACRES    │
│       varchar        │                  varchar                  │      varchar      │     varchar     │   varchar   │
├──────────────────────┼───────────────────────────────────────────┼───────────────────┼─────────────────┼─────────────┤
│ BLM                  │ United States Bureau of Land Management   │ Open Access       │ San Bernardino  │ 999.978904  │
│ Lassen National Fo…  │ United States Forest Service              │ Open Access       │ Shasta          │ 999.4542589 │
│ Unnamed site - Cal…  │ California Tahoe Conservancy              │ No Public Access  │ Placer          │ 999.4033232 │
│ Los Padres Nationa…  │ United States Forest Service              │ Open Access       │ Ventura         │ 998.7598506 │
│ Irvine Ranch Open …  │ Orange,

In [6]:
con.sql('''
    DESCRIBE cpad
''')

┌───────────────┬─────────────┬─────────┬─────────┬─────────┬───────┐
│  column_name  │ column_type │  null   │   key   │ default │ extra │
│    varchar    │   varchar   │ varchar │ varchar │ varchar │ int32 │
├───────────────┼─────────────┼─────────┼─────────┼─────────┼───────┤
│ OBJECTID      │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ NAME          │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ AGENCY        │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ AGNCY_TYP     │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ ACCESS        │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ LAND_WATER    │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ COUNTY        │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ ACRES         │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ DATE_REVIS    │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ YR_EST        │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ GAP1_acres    │ VA

In [24]:
con.sql('''
    SELECT COUNTY, ROUND(SUM(CAST(ACRES AS DOUBLE)),0) as SUM_ACRES
    FROM cpad
    GROUP BY COUNTY
    ORDER BY SUM_ACRES DESC
''')

┌────────────────┬───────────┐
│     COUNTY     │ SUM_ACRES │
│    varchar     │  double   │
├────────────────┼───────────┤
│ San Bernardino │ 8520747.0 │
│ Inyo           │ 5985768.0 │
│ Riverside      │ 2948155.0 │
│ Siskiyou       │ 2579078.0 │
│ Mono           │ 1863675.0 │
│ Lassen         │ 1771334.0 │
│ Modoc          │ 1716097.0 │
│ Imperial       │ 1575202.0 │
│ Trinity        │ 1564125.0 │
│ Tulare         │ 1558949.0 │
│   ·            │      ·    │
│   ·            │      ·    │
│   ·            │      ·    │
│ Santa Cruz     │   85694.0 │
│ Yuba           │   82869.0 │
│ Solano         │   74603.0 │
│ Sacramento     │   70849.0 │
│ Yolo           │   69666.0 │
│ Stanislaus     │   55134.0 │
│ San Joaquin    │   30776.0 │
│ Kings          │   23855.0 │
│ Sutter         │   14931.0 │
│ San Francisco  │    5850.0 │
├────────────────┴───────────┤
│     58 rows (20 shown)     │
└────────────────────────────┘

In [25]:
import leafmap
m = leafmap.Map()
m.add_shp("D:/OneDrive/Documents/ArcGIS/Projects/Tutorials/Chapter7/CPAD_2020b_Units.shp", layer_name="cpad")
m.set_center(-119.4179, 36.7783, zoom=6)
m

Map(center=[36.7783, -119.4179], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', '…

In [48]:
# Load the Inyo County and cpad shapefiles
inyo = gpd.read_file('C:/Users/vance/Downloads/inyo/inyo.shp')
cpad = gpd.read_file("D:/OneDrive/Documents/ArcGIS/Projects/Tutorials/Chapter7/CPAD_2020b_Units.shp")

In [63]:
# Ensure both GeoDataFrames are in the same coordinate reference system
inyo = inyo.to_crs(cpad.crs)

# Perform the intersection
intersection = gpd.overlay(cpad, inyo, how='intersection')

# Create a map
m = leafmap.Map()

# Add the intersection to the map
m.add_gdf(intersection, layer_name="intersection")

# Center the map
m.set_center(-117.3921, 36.5531, zoom=8)

# Display the map
m

Map(center=[36.5531, -117.3921], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', '…

In [ ]:
# Not sure if this will work
con.sql("""
        CREATE TABLE IF NOT EXISTS inyo as
        SELECT ROUND(SUM(CAST(area AS DOUBLE)),0) as TOTAL_AREA
        FROM intersection
""")